## GNFR A - Temporal profiles
Based on power statistics from relevant powerplants.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os 
import pandas as pd

In [ ]:
# import files
folder = '../data/GNFR A/power statistics/'

file_dict = {'HKW_Nord': 'Heizkraftwerk_M_nchen_Nord_2_201901010000_202412312359_Stunde.csv', 
             'HKW_Süd1': 'GuD_M_nchen_S_d_1_201901010000_202412312359_Stunde.csv',
             'HKW_Süd2': 'GuD_M_nchen_S_d_2_201901010000_202412312359_Stunde.csv',
}

In [20]:
def load_file(file_name, folder=folder):
    """Load a CSV file and return a DataFrame."""
    df = pd

    df = pd.read_csv(folder + file_name, sep=';', decimal=',')
    df.index = pd.to_datetime(df['Datum von'], format="%d.%m.%Y %H:%M")
    df.drop(columns=['Datum von', 'Datum bis'], inplace=True)

    # convert to numeric columns
    for c in df.columns:
        if df[c].dtype == 'object':
            df[c] = df[c].str.replace(',', '.')
        else:
            df[c] = df[c].astype(float)
            
    df = df.apply(pd.to_numeric, errors='coerce')
    df.fillna(0, inplace=True)
    return df

In [23]:
hkw_nord = load_file(file_dict['HKW_Nord'])
hkw_süd = load_file(file_dict['HKW_Süd1'])
hkw_süd = pd.concat([hkw_süd, load_file(file_dict['HKW_Süd2'])], axis=1)

# Prepare timeprofiles

**Format:**<br>
year, component, timestamp, scaling_factor

In [ ]:
# time profiles for HKW Nord
for y in hkw_nord.index.year.unique():
    df = hkw_nord[hkw_nord.index.year == y].copy()
    df = pd.DataFrame(data = {'scaling factor': df.sum(axis=1) / df.sum(axis=1).mean()})
    df['year'] = y
    df['component'] = 'all'
    df['timestamp'] = df.index
    df[['year', 'component', 'timestamp', 'scaling factor']]\
        .to_csv(f'../data/GNFR A/timeprofiles/GNFR_A_HKW_Nord_{y}.csv', index=False)

In [ ]:
# time profiles for HKW Süd
for y in hkw_süd.index.year.unique():
    df = hkw_süd[hkw_süd.index.year == y].copy()
    df = pd.DataFrame(data = {'scaling factor': df.sum(axis=1) / df.sum(axis=1).mean()})
    df['year'] = y
    df['component'] = 'all'
    df['timestamp'] = df.index
    df[['year', 'component', 'timestamp', 'scaling factor']]\
        .to_csv(f'../data/GNFR A/timeprofiles/GNFR_A_HKW_Süd_{y}.csv', index=False)